## Lab 04

## Zadanie 1 (Igor Sieradzki)

Funkcja <t>flow_rate</i> do modyfikacji

In [3]:
from prometheus_client import Counter


def flow_rate(weight, time):
    """ Funkcja wylicza ile wagi produktu przybyło/ubyło w jednostce czasu """
    return weight / time

weight = 0.5
time = 3

In [4]:
flow = flow_rate(weight, time)
print("{0:.3} kg per second".format(flow))

0.167 kg per second


In [ ]:
Zmodyfikuj funckje <i>flow_rate</i> tak, aby poniższe wywołania działały poprawnie.

In [5]:
def flow_rate(weight, time, period = 1, units_per_kg = 1):
    """ Funkcja wylicza ile wagi produktu przybyło/ubyło w jednostce czasu """
    return (weight / time) * units_per_kg * period

weight = 0.5
time = 3

In [6]:
flow = flow_rate(weight, time, period=60, units_per_kg=1000)
print("{} grams per minute".format(flow))

10000.0 grams per minute


In [7]:
flow = flow_rate(weight, time, period=1, units_per_kg=1)
print("{0:.3} kg per second".format(flow))

0.167 kg per second


In [8]:
flow = flow_rate(weight, time)
print("{0:.3} grams per minute".format(flow))

0.167 grams per minute


Wyjaśnić czemu poniższe rzucanie błędu jest porządane

In [9]:
try:
    flow = flow_rate(weight, time, "3600", 2.2)  #np do kontroli typów argumentów
except TypeError:
    print(True)

True


## Zadanie 2 (Igor Sieradzki)

Dopisz definicje dekoratora _timeit_, który wypisze na wyjscie standardowe czas wywołania udekorowanej funckji np.

```
@timeit
def foo(x):
    return x**2
    
r = foo(2)
```
wypize: <br>
    `Function foo took: 0.00001 seconds` <br>
oraz zwróci 4

In [10]:
import time

def timeit(func):
    """ Wypisuje czas wywołania udekorowanej funckji """
    start = time.perf_counter()
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    end = time.perf_counter()
    print( f"{func.__name__} took {end - start} seconds")
    return wrapper

@timeit
def squares_list(n):
    squares = []
    for i in range(n):
        squares.append(i ** 2)
    return squares

@timeit
def squares_comprehension(n):
    return [i ** 2 for i in range(n)]

@timeit
def squares_map(n):
    return map(lambda x: x**2, range(n))

n = 1000000
l = squares_list(n)
c = squares_comprehension(n)
m = squares_map(n)

squares_list took 6.00004568696022e-07 seconds
squares_comprehension took 5.00003807246685e-07 seconds
squares_map took 3.00002284348011e-07 seconds


## Zadanie 3 (Igor Sieradzki)

Dopisz definicje dekoratora _derivate_ wg. instrukcji w _docstringu_

In [1]:
import sys
sys.float_info.epsilon # epsilon maszynowy


def derivate(epsilon=None):
     """
     Zwraca pochodną funkcji w punkcie, wg. wzoru f'(x) = [f(x+h) - f(x)]/h,
     gdzie h jest parametrem dekoratora, jeśli nie zostanie podany, należy przyjąć 1000 * epsilon maszynowy
     """
     if epsilon is None:
         h = 1000 * sys.float_info.epsilon
     else:
         h = epsilon

     def decorator(f):
        def wrapper(x):
            return (f(x + h) - f(x)) / h
        return wrapper
     return decorator



@derivate(0.01)
def f(x):
    return x*x

@derivate(0.00001)
def g(x):
    return x*x*x+3

def test(a, b, eps=1):
    return abs(round(a)-round(b)) < eps

print(test(f(100), 200.0))
print(round(f(0)) == 0.0)

print(test(g(100), 30000.0))
print(round(g(0)) == 0.0)

from random import random
for x in [random()*1000. for _ in range(20)]:
    print(f(x), 2*x, '\t', test(f(x), 2*x))
    print(g(x), 3*x**2, '\t', test(g(x), 3*x**2))

True
True
True
True
367.4646803556243 367.4546803564822 	 True
101267.21216365694 101267.20658691338 	 True
1916.4935341686942 1916.483534164517 	 True
2754681.873321533 2754681.8525427883 	 True
1128.088450297946 1128.078450298397 	 True
954420.7543134688 954420.7425207248 	 True
1506.6553835175 1506.6453835250347 	 True
1702485.245466232 1702485.2337729745 	 True
276.9028140195587 276.89281401977195 	 True
57502.226997166865 57502.222841841016 	 True
438.0877587485884 438.0777587487268 	 True
143934.09863114357 143934.09203273075 	 True
839.8194684734335 839.8094684769848 	 True
528959.9671959877 528959.9575076968 	 True
1897.7303682942875 1897.7203683022483 	 True
2701006.9608688354 2701006.947201916 	 True
1757.2180039598607 1757.2080039565355 	 True
2315834.9990844727 2315834.9768766835 	 True
1146.4702711906284 1146.4602711900545 	 True
985778.3794403075 985778.36506288 	 True
489.3847137085686 489.37471370952744 	 True
179615.71514606473 179615.70781371146 	 True
162.33982601543

## Zadanie 4 (Igor Sieradzki)

Dopisz definicje dekoratora _accepts_ wg. instrukcji w _docstringu_

In [11]:
def accepts(*types):
    """Sprawdza czy udekorowanej funckji zostały podane odpowiednie parametry zdefiniowane
       w argumentach dekoratora"""
    def decorator(fun):
        def wrapper(*args, **kwargs):
            for arg, t in zip(args, types):
                if type(arg) != t:
                    raise TypeError(f"illegal parameter type {type(arg)}")
            return fun(*args, **kwargs)
        return wrapper
    return decorator


@accepts(str)
def capitalize(word):
    return word[0].upper() + word[1:]

print(capitalize('ola') == 'Ola')

try:
    capitalize(2)
except TypeError:
    print(True)

@accepts(float, int)
def static_pow(base, exp):
    return base ** exp

print(static_pow(2., 2) == 4.)
print(static_pow(2., exp=2) == 4.)
print(static_pow(base=2., exp=2) == 4.)

try:
    static_pow('x', 10)
except TypeError as e:
    print(True)

try:
    static_pow(2, 2.2)
except TypeError:
    print(True)

True
True
True
True
True
True
True


## Zadanie 5 (Igor Sieradzki)

Dopisz definicje dekoratora _returns_ wg. instrukcji w _docstringu_

In [12]:
from re import split

def returns(*types):
    """Sprawdza czy udekorowana funkcja zwraca poprawne argumenty, zdefiniowane w parametrach dekoratora"""
    def decorator(fun):
        def wrapper(*args, **kwargs):
            result = fun(*args, **kwargs)
            for res, t in zip(result, types):
                if not isinstance(res, t):
                    raise TypeError(f"illegal parameter {type(res)}")
            return result
        return wrapper
    return decorator


@returns(str)
def str_only_identity(word):
    return word

print(str_only_identity('hello') == 'hello')

try:
    str_only_identity(10)
except TypeError:
    print(True)

@returns(int, int)
def split_indices(x):
    return x[0], x[1]

print(split_indices(x=[6,9]) == (6,9))

try:
    split('AB')
except TypeError:
    print(True)

True
True
True
True


## Zadanie 6 (Igor Sieradzki)
Stwórz dekorator cached służący do cachowania wywołań dowolnej funkcji, tzn. chcemy by:
* wywołanie funkcji z określonymi argumentami miało miejsce tylko raz
* funkcja mogła przyjmować dowolną liczbę nazwanych i nienazwanych argumentów
* nie musi reagować poprawnie na domyślne argumenty, tzn. wywołanie funkcji z domyślnymi argumentami a podanie dokładnie takich samych może być traktowane jako dwa różne wywołania
* na opakowanej funkcji można wywołać `.cache_reset()`, który usunie cache z pamięci
* wywołanie `.cache_status()` zwraca string z opisem w postaci: <br>
    `Function FUNCTION_NAME called X times, evaluated Y times`

In [18]:
from random import random

def cached():
    cache = {}
    calls = 0
    evaluated = 0

    def decorator(func):
        def wrapper(*args, **kwargs):
            nonlocal calls, evaluated
            record = (args, tuple(sorted(kwargs.items())))
            calls += 1

            if record not in cache:
                result = func(*args, **kwargs)
                cache[record] = result
                evaluated += 1
                return result
            return cache[record]

        def cache_reset():
            cache.clear()

        def cache_status():
            return f'Function {func.__name__} called {calls} times, evaluated {evaluated} times'

        wrapper.cache_status = cache_status
        wrapper.cache_reset = cache_reset
        return wrapper
    return decorator



@cached()
def foo(x, y=1, z=4):
    return random()

print(foo(3) == foo(3))
print(foo(4) == foo(4))
print(foo(3, z=-1, y=3) == foo(3, y=3, z=-1))
print(foo(3) != foo(x=3))
a = foo(3)
foo.cache_reset()
print(a != foo(3))
print(foo.cache_status() == 'Function foo called 10 times, evaluated 5 times')

True
True
True
True
True
True


## Zadanie 7 (Krzysztof Hajto)

Napisz dekorator który będzie robić n-krotne złożenie funkcji, gdzie n jest parametrem dekoratora

In [17]:
from functools import reduce

def zlozenie(n):
    def decorator(func):
        def wrapper(x):
            return reduce(lambda y, _: func(y), range(n), x)
        return wrapper
    return decorator

@zlozenie(3)
def f1(x):
    return x+1

@zlozenie(2)
def f2(x):
    return x*x

@zlozenie(5)
def f3(word):
    return "".join(chr(ord(l)+1) for l in word)

print(f1(2)==5)
print(f2(3)==81)
print(f3("alamakota")=="fqfrfptyf")

True
True
True


## Zadanie 8 (Krzysztof Hajto)

Python nie ma wbudowanej instrukcji switch. Ale posiada anonimowe funkcje oraz słowniki. Zaimplementuj poniższy switch w postaci słownika funkcji.
`
int my_function(x, y) {


    switch(x) {
        case 1: return y*y;
        case 2: return x+y;
        case 3: return x*y;
        case 4: return 0;
    }
}
`

PS. Nigdy nie róbcie tego w faktycznym kodzie :)

In [16]:
switch = {
    1: lambda x,y: y*y,
    2: lambda x,y: x+y,
    3: lambda x,y: x*y,
    4: lambda x,y: 0,
    }

def my_function(x, y):
    return switch[x](x,y)

print(my_function(1,3)==9)
print(my_function(2,4)==6)
print(my_function(3,1)==3)
print(my_function(4,9)==0)

True
True
True
True
